<a href="https://colab.research.google.com/github/IvanSalinas2000/Preprocesamiento-y-clasificacion-de-datos/blob/main/Tesina/ChatBotMedEval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# DESCARGA DE LOS MODELOS DESDE GOOGLE DRIVE (públicos)
import gdown

# Enlaces públicos convertidos a IDs
model_id = "1BRBRG-S_ntf2WoRzE-G2i9P6TOY0bYRW"
tokenizer_id = "1rXDHQcFfciloVV6sykazFZkI_HHaZAm6"

# Descargar los archivos ZIP
gdown.download(f"https://drive.google.com/uc?id={model_id}", "t5_chatbot_model.zip", quiet=False)
gdown.download(f"https://drive.google.com/uc?id={tokenizer_id}", "t5_chatbot_tokenizer.zip", quiet=False)

# Descomprimir los archivos sin subcarpetas
!unzip -o -j t5_chatbot_model.zip -d /content/t5_chatbot_model
!unzip -o -j t5_chatbot_tokenizer.zip -d /content/t5_chatbot_tokenizer

Downloading...
From (original): https://drive.google.com/uc?id=1BRBRG-S_ntf2WoRzE-G2i9P6TOY0bYRW
From (redirected): https://drive.google.com/uc?id=1BRBRG-S_ntf2WoRzE-G2i9P6TOY0bYRW&confirm=t&uuid=8b9362ec-36d9-45ef-8ec1-a6d43fbdd0c7
To: /content/t5_chatbot_model.zip
100%|██████████| 824M/824M [00:10<00:00, 81.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1rXDHQcFfciloVV6sykazFZkI_HHaZAm6
To: /content/t5_chatbot_tokenizer.zip
100%|██████████| 416k/416k [00:00<00:00, 69.7MB/s]


Archive:  t5_chatbot_model.zip
  inflating: /content/t5_chatbot_model/config.json  
  inflating: /content/t5_chatbot_model/tokenizer_config.json  
  inflating: /content/t5_chatbot_model/spiece.model  
  inflating: /content/t5_chatbot_model/generation_config.json  
  inflating: /content/t5_chatbot_model/special_tokens_map.json  
  inflating: /content/t5_chatbot_model/added_tokens.json  
  inflating: /content/t5_chatbot_model/training_args.bin  
  inflating: /content/t5_chatbot_model/model.safetensors  
Archive:  t5_chatbot_tokenizer.zip
  inflating: /content/t5_chatbot_tokenizer/tokenizer_config.json  
  inflating: /content/t5_chatbot_tokenizer/spiece.model  
  inflating: /content/t5_chatbot_tokenizer/special_tokens_map.json  
  inflating: /content/t5_chatbot_tokenizer/added_tokens.json  


In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

# Rutas descomprimidas
model_path = "/content/t5_chatbot_model"
tokenizer_path = "/content/t5_chatbot_tokenizer"

# Cargar el tokenizador
tokenizer = T5Tokenizer.from_pretrained(tokenizer_path)

# Cargar el modelo
model = T5ForConditionalGeneration.from_pretrained(model_path)

# Usar GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Poner el modelo en modo evaluación
model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32100, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32100, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [3]:
def generate_response_top_k_top_p(
    question, model, tokenizer, max_length=64, top_k=50, top_p=0.95, temperature=1.0
):
    # Formatear la pregunta
    formatted_question = f"Answer the following question: {question}"

    # Tokenizar entrada
    inputs = tokenizer(
        formatted_question,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=128,
    ).to(device)

    # Generar respuesta
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=max_length,
        do_sample=True,
        top_k=top_k,
        top_p=top_p,
        temperature=temperature,
        pad_token_id=tokenizer.pad_token_id,
    )

    # Decodificar salida
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [4]:
# Pregunta de ejemplo
question = "what type of cancer exist?"

# Generar respuesta
response = generate_response_top_k_top_p(question, model, tokenizer)

# Mostrar resultado
print("Pregunta:", question)
print("Respuesta:", response)

Pregunta: what type of cancer exist?
Respuesta: the exact cause of every cancer is not known. the disease is a process that destroys healthy tissue. many cancers cause pain, swelling, and/or pain. some cancers can even be recurrences, but they usually start in the uterus, and can affect other areas
